# Perform Node Classification on the WACV Dataset

**Overview of the dataset**
- The new dataset is in `~/GraphPCB/Graph-WACV/graphs/` and `/home/lantian/GraphPCB/Graph-FPIC/graphs/`
- The nodes have 4 types/categories:
    - 0-IC  the target
    - 1-DT (hard to distinguish from 0-IC)
    - 2-Diode (also hard to tell from 0-IC)
    - 3-Others (everything else, should be easier to distinguish)
- **Note that not all categories are presented in all the graphs (which means, there are some graphs that has less than 4 types of nodes)**

**Graph-WACV**
- The train/test sets contain 37/10 graphs each.
- The stats of the graphs are like this:
    - avg num of nodes in each graph are mostly around 50-600
    - avg num of node degrees are between 5.7-5.9
    - the diameter of the graph are a normal distribution centered at ~14

**Plans**
- Try some node classification that presents in this repo

In [ ]:
# imports
import torch
from graphsep.model import Model
from logger import PCB_Logger
from utils import *


# Utility Functions

# Dataset Loader

- Load GraphPCB graphs as DGLGraphs

In [3]:
# Set the home directory for the dataset
home_dir = os.path.expanduser("~")
dataset_dir = os.path.join(home_dir, "GraphPCB")
print("Dataset directory:", dataset_dir)

Dataset directory: /home/lantian/GraphPCB


In [2]:
class GraphPCBDataset:
    def __init__(self, data_dir, device='cpu', add_self_loops=False, to_bidirectional=True):
        """
        Initialize the dataset and convert graphs into DGLGraph format.

        Args:
            data_dir (str): Directory containing the `.pt` files with torch_geometric Data objects.
            device (str): Device to load the data onto ('cpu' or 'cuda').
            add_self_loops (bool): Whether to add self-loops to the graphs.
            to_bidirectional (bool): Whether to convert graphs to bidirectional.
        """
        print("Loading dataset...")
        self.device = device
        self.graphs = []
        self.graph_files = [f for f in os.listdir(data_dir) if f.endswith('.pt')]
        #self.idx_to_graph = {i: f for i, f in enumerate(self.graph_files)}
        # ramdomly shuffle the graph files
        np.random.shuffle(self.graph_files)
        self.num_node_features = None  # Initialize to None
        self.num_classes = None  # Initialize to None
        self.size = len(self.graph_files)

        # Load and convert each graph
        for filename in self.graph_files:
            if filename.endswith(".pt"):
                data = torch.load(os.path.join(data_dir, filename))  # Load torch_geometric Data
                # get the number of node features and the number of classes
                self.num_node_features = data.x.size(1)
                self.num_targets = data.y.max().item() + 1 if data.y is not None else 0
                
                dgl_graph = dgl.graph((data.edge_index[0], data.edge_index[1]), num_nodes=data.x.size(0))

                if to_bidirectional:
                    dgl_graph = dgl.to_bidirected(dgl_graph)
                if add_self_loops:
                    dgl_graph = dgl.add_self_loop(dgl_graph)

                                    # add node features and labels
                dgl_graph.ndata['x'] = data.x
                dgl_graph.ndata['y'] = data.y

                dgl_graph = dgl_graph.to(self.device)

                self.graphs.append(dgl_graph)
                # Set num_node_features and num_classes based on the first graph
                if self.num_node_features is None:
                    self.num_node_features = data.x.size(1)
                if self.num_classes is None and data.y is not None:
                    self.num_classes = data.y.max().item() + 1

        print(f"Loaded {len(self.graphs)} graphs.")

        self.num_node_features = data.x.size(1)
        self.num_targets = data.y.max().item() + 1 if data.y is not None else 0

    def __getitem__(self, idx):
        return {"graph": self.graphs[idx], "filename": self.graph_files[idx]}

    def __len__(self):
        return len(self.graphs)

# Inference

In [4]:
def inference(model, dataset):
    model.eval()
    all_preds, all_labels = [], []
    predictions = []

    for i in range(dataset.size):
        graph_file_name = dataset[i]['filename']

        with torch.no_grad():
            node_features = dataset[i]['graph'].ndata['x']
            y_true = dataset[i]['graph'].ndata['y']
            logits = model(graph=dataset[i]['graph'], x=node_features)
            preds = logits.argmax(dim=1)

        predictions.append({
            "graph_id": graph_file_name,
            "labels": preds.tolist()
        })

        all_preds.append(preds.cpu())
        all_labels.append(y_true.cpu())

    return torch.cat(all_preds).numpy(), torch.cat(all_labels).numpy(), predictions


# Training Function
- for every 10 epoch, save the model and evaluate on test set
- save the training log into a .txt
- save the last checkpoint
- save the predictions into a .json

In [5]:
def train_step(model, dataset, optimizer, scheduler, scaler, amp=False):
    total_loss = 0
    model.train()

    optimizer.zero_grad()
    
    for i in range(len(dataset)):
        with autocast(enabled=amp):
            graph = dataset[i]['graph']
            node_features = graph.ndata['x']
            logits = model(graph=graph, x=node_features)
            y = graph.ndata['y']

            loss = compute_loss(logits, y, num_classes=4, gamma=2.0, temperature=0.02, loss_weights=[1.0, 0.1])
            total_loss += loss

    scaler.scale(total_loss).backward()
    scaler.step(optimizer)
    scaler.update()
    
    scheduler.step()
    avg_loss = total_loss / len(dataset)
    return avg_loss


In [6]:
# the main function to train the model
def train_model(model, config):
    """
    Generic function to train different GNN models.
    """
    set_seed(42)
    train_dir = os.path.join(config['dataset_dir'], f"Graph-{config['dataset'].upper()}/graphs", "train")
    test_dir = os.path.join(config['dataset_dir'], f"Graph-{config['dataset'].upper()}/graphs", "test")
    train_dataset = GraphPCBDataset(data_dir=train_dir, device=config["device"], add_self_loops=False)
    test_dataset = GraphPCBDataset(data_dir=test_dir, device=config["device"], add_self_loops=False)

    logger = PCB_Logger(home_dir='/home/lantian/', config=config)

    
    torch.cuda.empty_cache()
    # ✅ Move model to device
    model = model.to(config["device"])
    
    # ✅ Define optimizer & scheduler
    parameter_groups = get_parameter_groups(model)
    optimizer = torch.optim.AdamW(parameter_groups, lr=config['learning_rate'], weight_decay=config['weight_decay'])
    scaler = GradScaler(enabled=config['amp'])
    scheduler = get_lr_scheduler_with_warmup(optimizer=optimizer, num_warmup_steps=config['num_warmup_steps'],
                                            num_steps=config['num_epochs'], warmup_proportion=config['warmup_proportion'])

    # ✅ Training Loop
    for epoch in range(config["num_epochs"]):
        avg_loss = train_step(model, train_dataset, optimizer, scheduler, scaler, config["amp"])
        logger.log(f"Epoch {epoch + 1:03d}, Loss: {avg_loss:.10f}")
        for name, param in model.named_parameters():
            if not param.requires_grad:
                print(f"Parameter with requires_grad=False: {name}")

        # ✅ Evaluate every 10 epochs OR at the last epoch
        if (epoch + 1) % 10 == 0 or (epoch + 1 == config["num_epochs"]):
            all_preds, all_labels, predictions = inference(model, test_dataset)
            metrics = compute_metrics(all_preds, all_labels)
            logger.update_metrics(metrics, predictions)
    
    logger.finish_run()
    # save only the final model
    checkpoint_path = os.path.join(logger.checkpoint_dir, f"model_epoch_{epoch + 1}_{avg_loss:.4f}.pth")
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'metrics': metrics,
    }, checkpoint_path)
    logger.log(f"✅ Model checkpoint saved to {checkpoint_path}")

    return model, metrics, logger.checkpoint_dir

## Config and Args

In [7]:
# Basic configuration 
config = {
    "experiment_name": "",
    "dataset_dir": "/home/lantian/GraphPCB/",
    "home_dir": "/home/lantian/",
    "dataset": "fpic",
    "batch_size": 1,

    # model architecture
    "model": "GT-sep",
    "num_layers": 2,
    "input_dim": 1024,
    "hidden_dim": 256,
    "hidden_dim_multiplier": 1.0,
    "output_dim": 4,
    "num_heads": 8,
    "normalization": "LayerNorm", # choices=['None', 'LayerNorm', 'BatchNorm']

    # regularization
    "dropout": 0.5,
    "weight_decay": 1e-2,

    # training parameters
    "learning_rate": 1e-4,
    "num_epochs": 200,
    "num_warmup_steps": None,
    "warmup_proportion": 0.1,

    # node feature augmentation
    "use_sgc_features": False,
    "use_identity_features": False,
    "use_adjacency_features": False,
    "do_not_use_original_features": False,

    # system & misc
    "device": "cuda:0",
    "amp": False,
    "verbose": False
}

In [8]:
run_num = 0

config.update({
    "dataset": "fpic",

    # model architecture
    "model": "GT-sep",
    "num_layers": 2,
    "hidden_dim": 256,
    "hidden_dim_multiplier": 1.0,
    "num_heads": 8,
    "normalization": "LayerNorm", # choices=['None', 'LayerNorm', 'BatchNorm']

    # regularization
    "dropout": 0.5,
    "weight_decay": 1e-2,

    # training parameters
    "learning_rate": 1e-4,
    "num_epochs": 200,
    "num_warmup_steps": None,
    "warmup_proportion": 0.1,
})

config["experiment_name"] = f"{config['model']}-l{config['num_layers']}_{run_num}-NLL"

# Set seed before training
set_seed(42)

# Instantiate the model
model = Model(model_name=config["model"],
            num_layers=config["num_layers"],
            input_dim=config["input_dim"],
            hidden_dim=config["hidden_dim"],
            output_dim=config["output_dim"],
            hidden_dim_multiplier=config["hidden_dim_multiplier"],
            num_heads=config["num_heads"],
            normalization=config["normalization"],
            dropout=config["dropout"])

model, metrics, checkpoint_dir = train_model(model, config)

Loading dataset...
Loaded 115 graphs.
Loading dataset...
Loaded 47 graphs.
Checkpoint directory: /home/lantian/PCB_Analysis/FPIC-trained/GT-sep-l2_0-NLL
Experiment Configuration:
experiment_name: GT-sep-l2_0-NLL
dataset_dir: /home/lantian/GraphPCB/
home_dir: /home/lantian/
dataset: fpic
batch_size: 1
model: GT-sep
num_layers: 2
input_dim: 1024
hidden_dim: 256
hidden_dim_multiplier: 1.0
output_dim: 4
num_heads: 8
normalization: LayerNorm
dropout: 0.5
weight_decay: 0.01
learning_rate: 0.0001
num_epochs: 200
num_warmup_steps: None
warmup_proportion: 0.1
use_sgc_features: False
use_identity_features: False
use_adjacency_features: False
do_not_use_original_features: False
device: cuda:0
amp: False
verbose: False
Using device: cuda:0
Loading dataset: fpic
Results will be saved to /home/lantian/PCB_Analysis/FPIC-trained/GT-sep-l2_0-NLL.
Epoch 001, Loss: 1.6141564846
Epoch 002, Loss: 1.5837553740
Epoch 003, Loss: 1.5704184771
Epoch 004, Loss: 1.5491716862
Epoch 005, Loss: 1.5279779434
Epoch 00

/home/lantian/PCB_Analysis_NIPS_code_submit/utils.py:83: RuntimeWarning: invalid value encountered in divide
  precision = np.mean(TP / (TP + FP)) if np.sum(TP + FP) > 0 else 0


Epoch 021, Loss: 1.2352473736
Epoch 022, Loss: 1.2102465630
Epoch 023, Loss: 1.2024562359
Epoch 024, Loss: 1.1844984293
Epoch 025, Loss: 1.1646085978
Epoch 026, Loss: 1.1465977430
Epoch 027, Loss: 1.1129816771
Epoch 028, Loss: 1.0842778683
Epoch 029, Loss: 1.0452280045
Epoch 030, Loss: 1.0367926359
  F1-Score (macro): 0.3918981066
  Weighted F1: 0.9129668560
  Subset F1-Score (3-class): 0.0000000000
  F1 per class: [0.611032531824611, 0.0, 0.0, 0.9565598946906537]
  Precision per class: [0.45, 0.0, 0.0, 0.9796285200718994]
  Recall per class: [0.9515418502202643, 0.0, 0.0, 0.9345527293512432]
  Confusion Matrix:
[[216, 0, 0, 11], [30, 0, 0, 22], [5, 0, 0, 35], [229, 0, 0, 3270]]


/home/lantian/PCB_Analysis_NIPS_code_submit/utils.py:83: RuntimeWarning: invalid value encountered in divide
  precision = np.mean(TP / (TP + FP)) if np.sum(TP + FP) > 0 else 0


Epoch 031, Loss: 1.0070148706
Epoch 032, Loss: 0.9839547873
Epoch 033, Loss: 0.9547399282
Epoch 034, Loss: 0.9367415309
Epoch 035, Loss: 0.9146968722
Epoch 036, Loss: 0.8711636066
Epoch 037, Loss: 0.8608847260
Epoch 038, Loss: 0.8353277445
Epoch 039, Loss: 0.8055003881
Epoch 040, Loss: 0.8027477860
  F1-Score (macro): 0.5292690276
  Weighted F1: 0.9149428440
  Subset F1-Score (3-class): 0.6688981051
  F1 per class: [0.6782334384858043, 0.30409356725146197, 0.18705035971223022, 0.9476987447698745]
  Precision per class: [0.5282555282555282, 0.2184873949579832, 0.13131313131313133, 0.9931099279674287]
  Recall per class: [0.947136563876652, 0.5, 0.325, 0.906258931123178]
  Confusion Matrix:
[[215, 6, 0, 6], [10, 26, 15, 1], [0, 12, 13, 15], [182, 75, 71, 3171]]
Epoch 041, Loss: 0.7931294441
Epoch 042, Loss: 0.7535772324
Epoch 043, Loss: 0.7435692549
Epoch 044, Loss: 0.7471035123
Epoch 045, Loss: 0.7310517430
Epoch 046, Loss: 0.7264531255
Epoch 047, Loss: 0.6988151670
Epoch 048, Loss: 0.6

# Run Multiple
- Run multiple experiments for all the settings
    - `model` = `GT-sep` or `GAT-sep`
    - `num_layer` from 1 to 5

In [91]:
# Define the settings to iterate over
dataset_variants = ['FPIC', 'WACV']
model_variants = ['GT-sep', 'GAT-sep']
layer_counts = range(1, 6)  # From 1 to 5 layers

# Iterate over each combination of model and layer count
for dataset in dataset_variants:
    for model_name in model_variants:
        for num_layers in layer_counts:
            # Update experiment name
            exp_name = f"{model_name}-l{num_layers}_{run_num}-NLL"
            
            # Update configuration
            config.update({
                "dataset": dataset,
                "experiment_name": exp_name,
                "model": model_name,
                "num_layers": num_layers,
            })
            
            # Instantiate the model
            model = Model(
                model_name=config["model"],
                num_layers=config["num_layers"],
                input_dim=config["input_dim"],
                hidden_dim=config["hidden_dim"],
                output_dim=config["output_dim"],
                hidden_dim_multiplier=config["hidden_dim_multiplier"],
                num_heads=config["num_heads"],
                normalization=config["normalization"],
                dropout=config["dropout"]
            )
            
            # Train the model
            print(f"Training {exp_name}...")
            model, metrics, checkpoint_dir = train_model(model, config)
            print(f"Finished training {exp_name}. Metrics: {metrics}")

Training GT-sep-l1_0-NLL...
Loading dataset...
Loaded 115 graphs.
Loading dataset...
Loaded 47 graphs.
Checkpoint directory: /home/lantian/PCB_Analysis/FPIC-trained/GT-sep-l1_0-NLL
Experiment Configuration:
experiment_name: GT-sep-l1_0-NLL
dataset_dir: /home/lantian/GraphPCB/
home_dir: /home/lantian/
dataset: FPIC
batch_size: 1
model: GT-sep
num_layers: 1
input_dim: 1024
hidden_dim: 256
hidden_dim_multiplier: 1.0
output_dim: 4
num_heads: 8
normalization: LayerNorm
dropout: 0.5
weight_decay: 0.01
learning_rate: 0.0001
num_epochs: 200
num_warmup_steps: None
warmup_proportion: 0.1
use_sgc_features: False
use_identity_features: False
use_adjacency_features: False
do_not_use_original_features: False
device: cuda:0
amp: False
verbose: False
Using device: cuda:0
Loading dataset: FPIC
Results will be saved to /home/lantian/PCB_Analysis/FPIC-trained/GT-sep-l1_0-NLL.
Epoch 001, Loss: 1.5333482027
Epoch 002, Loss: 1.5191663504
Epoch 003, Loss: 1.4958550930
Epoch 004, Loss: 1.4726063013
Epoch 005,

/tmp/ipykernel_2913790/2735098002.py:20: RuntimeWarning: invalid value encountered in divide
  precision = np.mean(TP / (TP + FP)) if np.sum(TP + FP) > 0 else 0


Epoch 011, Loss: 1.3586262465
Epoch 012, Loss: 1.3665509224
Epoch 013, Loss: 1.3518123627
Epoch 014, Loss: 1.3304561377
Epoch 015, Loss: 1.3365768194
Epoch 016, Loss: 1.3084373474
Epoch 017, Loss: 1.2718286514
Epoch 018, Loss: 1.2425658703
Epoch 019, Loss: 1.2280911207
Epoch 020, Loss: 1.1901402473
  F1-Score (macro): 0.3946885464
  Weighted F1: 0.9169786828
  Subset F1-Score (3-class): 0.0000000000
  F1 per class: [0.618287373004354, 0.0, 0.0, 0.9604668125455872]
  Precision per class: [0.461038961038961, 0.0, 0.0, 0.9809296781883194]
  Recall per class: [0.9383259911894273, 0.0, 0.0, 0.940840240068591]
  Confusion Matrix:
[[213, 0, 0, 14], [36, 0, 0, 16], [6, 0, 0, 34], [207, 0, 0, 3292]]


/tmp/ipykernel_2913790/2735098002.py:20: RuntimeWarning: invalid value encountered in divide
  precision = np.mean(TP / (TP + FP)) if np.sum(TP + FP) > 0 else 0


Epoch 021, Loss: 1.1690208912
Epoch 022, Loss: 1.1585134268
Epoch 023, Loss: 1.1311612129
Epoch 024, Loss: 1.1144098043
Epoch 025, Loss: 1.0922484398
Epoch 026, Loss: 1.0637972355
Epoch 027, Loss: 1.0263955593
Epoch 028, Loss: 0.9917170405
Epoch 029, Loss: 0.9616097212
Epoch 030, Loss: 0.9335029721
  F1-Score (macro): 0.5012347821
  Weighted F1: 0.9218394867
  Subset F1-Score (3-class): 0.5032382299
  F1 per class: [0.6235632183908046, 0.42222222222222217, 0.0, 0.9591536879224215]
  Precision per class: [0.4626865671641791, 0.5, 0.0, 0.9869972785001512]
  Recall per class: [0.9559471365638766, 0.36538461538461536, 0.0, 0.9328379537010575]
  Confusion Matrix:
[[217, 3, 0, 7], [26, 19, 3, 4], [6, 2, 0, 32], [220, 14, 1, 3264]]
Epoch 031, Loss: 0.8924689293
Epoch 032, Loss: 0.8919787407
Epoch 033, Loss: 0.8632055521
Epoch 034, Loss: 0.8326552510
Epoch 035, Loss: 0.7995638847
Epoch 036, Loss: 0.7711047530
Epoch 037, Loss: 0.7665898800
Epoch 038, Loss: 0.7583646178
Epoch 039, Loss: 0.736572

...

Epoch 021, Loss: 1.1433092356
Epoch 022, Loss: 1.1044341326
Epoch 023, Loss: 1.0993378162
Epoch 024, Loss: 1.0671294928
Epoch 025, Loss: 1.0672349930
Epoch 026, Loss: 1.0673440695
Epoch 027, Loss: 1.0335688591
Epoch 028, Loss: 1.0692304373
Epoch 029, Loss: 0.9913393259
Epoch 030, Loss: 1.0093786716
  F1-Score (macro): 0.3235873295
  Weighted F1: 0.9355109764
  Subset F1-Score (3-class): 0.0000000000
  F1 per class: [0.329004329004329, 0.0, 0.0, 0.9653449890727442]
  Precision per class: [0.23529411764705882, 0.0, 0.0, 0.977449947312961]
  Recall per class: [0.5467625899280576, 0.0, 0.0, 0.9535361842105263]
  Confusion Matrix:
[[76, 0, 0, 63], [9, 0, 0, 32], [12, 0, 0, 12], [226, 0, 0, 4638]]


/tmp/ipykernel_2913790/2735098002.py:20: RuntimeWarning: invalid value encountered in divide
  precision = np.mean(TP / (TP + FP)) if np.sum(TP + FP) > 0 else 0


Epoch 031, Loss: 0.9729492664
Epoch 032, Loss: 0.9446874261
Epoch 033, Loss: 0.9504622817
Epoch 034, Loss: 0.9031908512
Epoch 035, Loss: 0.8727291226
Epoch 036, Loss: 0.8416967988
Epoch 037, Loss: 0.8422325850
Epoch 038, Loss: 0.8323562741
Epoch 039, Loss: 0.7844907045
Epoch 040, Loss: 0.7544664741
  F1-Score (macro): 0.3329375818
  Weighted F1: 0.9299625608
  Subset F1-Score (3-class): 0.0000000000
  F1 per class: [0.3734567901234568, 0.0, 0.0, 0.9582935370900988]
  Precision per class: [0.23772102161100198, 0.0, 0.0, 0.9903487606931345]
  Recall per class: [0.8705035971223022, 0.0, 0.0, 0.9282483552631579]
  Confusion Matrix:
[[121, 0, 0, 18], [21, 0, 0, 20], [18, 0, 0, 6], [349, 0, 0, 4515]]


/tmp/ipykernel_2913790/2735098002.py:20: RuntimeWarning: invalid value encountered in divide
  precision = np.mean(TP / (TP + FP)) if np.sum(TP + FP) > 0 else 0


Epoch 041, Loss: 0.7454030514
Epoch 042, Loss: 0.7631440163
Epoch 043, Loss: 0.7048526406
Epoch 044, Loss: 0.6622484326
Epoch 045, Loss: 0.7155339718
Epoch 046, Loss: 0.6303549409
Epoch 047, Loss: 0.6469284892
Epoch 048, Loss: 0.6400987506
Epoch 049, Loss: 0.6264054775
Epoch 050, Loss: 0.6190595031
  F1-Score (macro): 0.4048810395
  Weighted F1: 0.9393231933
  Subset F1-Score (3-class): 0.4143629144
  F1 per class: [0.4719101123595505, 0.1333333333333333, 0.0504201680672269, 0.9638605442176871]
  Precision per class: [0.3189873417721519, 0.14705882352941177, 0.031578947368421054, 0.9977992957746479]
  Recall per class: [0.9064748201438849, 0.12195121951219512, 0.125, 0.9321546052631579]
  Confusion Matrix:
[[126, 5, 2, 6], [19, 5, 14, 3], [15, 5, 3, 1], [235, 19, 76, 4534]]
Epoch 051, Loss: 0.6336783767
Epoch 052, Loss: 0.5823305249
Epoch 053, Loss: 0.5877015591
Epoch 054, Loss: 0.5748950243
Epoch 055, Loss: 0.5659683943
Epoch 056, Loss: 0.5449629426
Epoch 057, Loss: 0.5617510676
Epoch